In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input/"))

# Any results you write to the current directory are saved as output.

In [ ]:
import torch.utils.data
import numpy as np
import pandas as pd
from pytorch_pretrained_bert import BertTokenizer, BertForSequenceClassification, BertAdam, BertConfig
from tqdm import tqdm_notebook as tqdm
device = torch.device('cuda')

In [ ]:
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    return np.array(all_tokens)

In [ ]:
MAX_SEQUENCE_LENGTH = 220
BATCH_SIZE = 8

BERT_MODEL_PATH = '../input/bert-pretrained-models/uncased_l-12_h-768_a-12/uncased_L-12_H-768_A-12/'
bert_config = BertConfig('../input/bertinference3/bert/bert_config.json')

tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None, do_lower_case=True)

In [ ]:
df = pd.read_csv('../input/quora-insincere-questions-classification/test.csv')
df['question_text'] = df['question_text'].astype(str)
x_test = convert_lines(df['question_text'], MAX_SEQUENCE_LENGTH, tokenizer)

In [ ]:
model = BertForSequenceClassification(bert_config, num_labels=1)
model.load_state_dict(torch.load("../input/bertinference3/bert/bert_pytorch.bin"))
model.to(device)
for param in model.parameters():
    param.require_grad = False
model.eval()

In [ ]:
test_preds = np.zeros((len(x_test)))
test = torch.utils.data.TensorDataset(torch.tensor(x_test, dtype=torch.long))
test_loader = torch.utils.data.DataLoader(test, batch_size=9, shuffle=False)
tk0 = tqdm(test_loader)
for i, (x_batch,) in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch > 0).to(device), labels=None)
    test_preds[i * 9:(i + 1) * 9] = pred[:, 0].detach().cpu().squeeze().numpy()

test_pred = torch.sigmoid(torch.tensor(test_preds)).numpy().ravel()

In [ ]:
val = test_pred.copy()

In [ ]:
def ro(val):
    return [1 if i>0.37 else 0 for i in val]
a = ro(val)

In [ ]:

submission = pd.DataFrame({'qid':df['qid'], 
                   'prediction':a})
submission.to_csv('submission.csv', index=False)